# h2oai groupby benchmarks

In [4]:
import coiled
import dask
import dask.dataframe as dd

import pandas as pd

In [2]:
cluster = coiled.Cluster(name="demo-cluster", n_workers=5)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:226: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-96026-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-96026-firewall -> coiled-dask-matthew24-96026-firewall]
Created FW rules: coiled-dask-matthew24-96026-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-96026-cluster-firewall-d7fa40 -> coiled-dask-matthew24-96026-cluster-firewall]
Created scheduler VM: coiled-dask-matthew24-96026-scheduler (type: c5.large, ip: ['3.227.12.33'])


In [5]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2021.11.2     | 2021.12.0     | 2021.12.0     |
| distributed | 2021.11.2     | 2021.12.0     | 2021.12.0     |
| numpy       | 1.21.4        | 1.22.0        | 1.22.0        |
| pandas      | 1.3.4         | 1.3.5         | 1.3.5         |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [8]:
src_grp_1e7_csv = "s3://coiled-datasets/h2o/G1_1e7_1e2_0_0/csv/G1_1e7_1e2_0_0.csv"
src_grp_1e7_parquet = "s3://coiled-datasets/h2o/G1_1e7_1e2_0_0/parquet"

In [9]:
x = dd.read_csv(
    src_grp_1e7_csv,
    dtype={
        "id1": "category",
        "id2": "category",
        "id3": "category",
        "id4": "Int32",
        "id5": "Int32",
        "id6": "Int32",
        "v1": "Int32",
        "v2": "Int32",
        "v3": "float64",
    },
)

In [10]:
print(x.head())

     id1    id2           id3  id4  id5    id6  v1  v2         v3
0  id016  id016  id0000042202   15   24   5971   5  11  37.211254
1  id039  id045  id0000029558   40   49  39457   5   4  48.951141
2  id047  id023  id0000071286   68   20  74463   2  14  60.469241
3  id043  id057  id0000015141   32   43  63743   1  15   7.692145
4  id054  id040  id0000011083    9   25  16920   2   9  22.863525


In [11]:
len(x)

10000000

In [12]:
x.npartitions

8

In [13]:
x.memory_usage(deep=True).compute()

Index         1024
id1       10082944
id2       10082944
id3      112108332
id4       50000000
id5       50000000
id6       50000000
v1        50000000
v2        50000000
v3        80000000
dtype: int64

In [14]:
ddf = x.repartition(partition_size="100MB")

In [12]:
ddf.npartitions

8

In [13]:
ddf.to_parquet("/Users/powers/Documents/tmp/data/h2o/parquet/G1_1e7_1e2_0_0")

(None,)

## q1: sum v1 by id1

In [15]:
%%time
x.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 90.3 ms, sys: 7.67 ms, total: 97.9 ms
Wall time: 10.6 s


,v1
id1,
id016,298268
id039,299711
id047,298115
id043,299557
id054,299408
...,...
id008,300392
id062,299313
id011,300188


In [22]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id1", "v1"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [23]:
%%time
ddf.groupby("id1", dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 41.4 ms, sys: 4.89 ms, total: 46.3 ms
Wall time: 1.31 s


,v1
id1,
id016,298268
id039,299711
id047,298115
id043,299557
id054,299408
...,...
id008,300392
id062,299313
id011,300188


## q2: sum v1 by id1:id2

In [27]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id1", "id2", "v1"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [28]:
%%time
ddf.groupby(["id1", "id2"], dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 46.1 ms, sys: 4.54 ms, total: 50.6 ms
Wall time: 2.93 s


v1
id1   id2        
id016 id016  3072
      id045  3160
      id023  2931
      id057  2984
      id040  2770
...           ...
id096 id028  3017
      id082  3184
      id065  3159
      id013  2903
      id089  3002

[10000 rows x 1 columns]

In [29]:
%%time
x.groupby(["id1", "id2"], dropna=False, observed=True).agg({"v1": "sum"}).compute()

CPU times: user 84.2 ms, sys: 7.76 ms, total: 91.9 ms
Wall time: 10.7 s


v1
id1   id2        
id016 id016  3072
      id045  3160
      id023  2931
      id057  2984
      id040  2770
...           ...
id096 id028  3017
      id082  3184
      id065  3159
      id013  2903
      id089  3002

[10000 rows x 1 columns]

## q3: sum v1 mean v3 by id3

In [30]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id3", "v1", "v3"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [31]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg(
    {"v1": "sum", "v3": "mean"}
).compute()

CPU times: user 111 ms, sys: 25.9 ms, total: 137 ms
Wall time: 5.29 s


,v1,v3
id3,,
id0000042202,281,53.198223
id0000029558,327,45.245052
id0000071286,322,49.842035
id0000015141,308,50.790698
id0000011083,307,47.676258
...,...,...
id0000009966,318,53.899987
id0000064729,298,48.259954
id0000084431,238,50.899993


In [32]:
%%time
x.groupby("id3", dropna=False, observed=True).agg({"v1": "sum", "v3": "mean"}).compute()

CPU times: user 145 ms, sys: 21 ms, total: 166 ms
Wall time: 12.3 s


,v1,v3
id3,,
id0000042202,281,53.198223
id0000029558,327,45.245052
id0000071286,322,49.842035
id0000015141,308,50.790698
id0000011083,307,47.676258
...,...,...
id0000009966,318,53.899987
id0000064729,298,48.259954
id0000084431,238,50.899993


## q4: mean v1:v3 by id4

In [33]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id4", "v1", "v2", "v3"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [34]:
%%time
ddf.groupby("id4", dropna=False, observed=True).agg(
    {"v1": "mean", "v2": "mean", "v3": "mean"}
).compute()

CPU times: user 34.2 ms, sys: 2.8 ms, total: 37 ms
Wall time: 1.26 s


,v1,v2,v3
id4,,,
1,3.001875,7.995427,49.983907
2,2.998208,7.996927,49.841268
3,3.000010,7.998429,49.880546
4,3.005033,8.008521,50.053042
5,2.999491,8.013990,50.005535
...,...,...,...
96,2.996474,7.983614,49.889232
97,2.995558,7.990926,50.038346
98,2.986841,8.012647,50.124801


In [35]:
%%time
x.groupby("id4", dropna=False, observed=True).agg(
    {"v1": "mean", "v2": "mean", "v3": "mean"}
).compute()

CPU times: user 93.5 ms, sys: 7.98 ms, total: 101 ms
Wall time: 10.3 s


,v1,v2,v3
id4,,,
1,3.001875,7.995427,49.983907
2,2.998208,7.996927,49.841268
3,3.00001,7.998429,49.880546
4,3.005033,8.008521,50.053042
5,2.999491,8.01399,50.005535
...,...,...,...
96,2.996474,7.983614,49.889232
97,2.995558,7.990926,50.038346
98,2.986841,8.012647,50.124801


## q5: sum v1:v3 by id6

In [36]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id6", "v1", "v2", "v3"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [37]:
%%time
ddf.groupby("id6", dropna=False, observed=True).agg(
    {"v1": "sum", "v2": "sum", "v3": "sum"}
).compute()

CPU times: user 63.5 ms, sys: 11.3 ms, total: 74.8 ms
Wall time: 1.88 s


,v1,v2,v3
id6,,,
1,278,759,4500.324131
2,296,803,5139.198430
3,233,622,4081.984864
4,282,877,5539.323994
5,267,705,3886.950520
...,...,...,...
99996,339,977,5925.965188
99997,262,745,4610.177984
99998,333,759,5134.606333


In [38]:
%%time
x.groupby("id6", dropna=False, observed=True).agg(
    {"v1": "sum", "v2": "sum", "v3": "sum"}
).compute()

CPU times: user 119 ms, sys: 18.6 ms, total: 138 ms
Wall time: 11.2 s


,v1,v2,v3
id6,,,
1,278,759,4500.324131
2,296,803,5139.198430
3,233,622,4081.984864
4,282,877,5539.323994
5,267,705,3886.950520
...,...,...,...
99996,339,977,5925.965188
99997,262,745,4610.177984
99998,333,759,5134.606333


## q6: Intentionally skipped cause it's commented out

## q7: max v1 - min v2 by id3

In [39]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id3", "v1", "v2"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [40]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 101 ms, sys: 19.6 ms, total: 121 ms
Wall time: 2.85 s


,range_v1_v2
id3,
id0000042202,4
id0000029558,4
id0000071286,4
id0000015141,4
id0000011083,4
...,...
id0000009966,4
id0000064729,4
id0000084431,4


In [41]:
%%time
x.groupby("id3", dropna=False, observed=True).agg({"v1": "max", "v2": "min"}).assign(
    range_v1_v2=lambda x: x["v1"] - x["v2"]
)[["range_v1_v2"]].compute()

CPU times: user 158 ms, sys: 18.1 ms, total: 176 ms
Wall time: 12.1 s


,range_v1_v2
id3,
id0000042202,4
id0000029558,4
id0000071286,4
id0000015141,4
id0000011083,4
...,...
id0000009966,4
id0000064729,4
id0000084431,4


## q8: largest two v3 by id6

In [42]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id6", "v1", "v2", "v3"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [43]:
%%time
ddf[~ddf["v3"].isna()][["id6", "v3"]].groupby("id6", dropna=False, observed=True).apply(
    lambda x: x.nlargest(2, columns="v3"), meta={"id6": "Int64", "v3": "float64"}
)[["v3"]].compute()

CPU times: user 309 ms, sys: 36.7 ms, total: 345 ms
Wall time: 24.8 s


v3
id6                     
9     708485   99.930063
      1233683  99.920439
19    82918    97.625727
      625200   97.036998
33    1183098  99.414996
...                  ...
99994 168013   96.185434
99998 136085   99.405711
      197174   99.278838
99999 667180   99.555205
      290571   97.386879

[200000 rows x 1 columns]

In [44]:
%%time
x[~x["v3"].isna()][["id6", "v3"]].groupby("id6", dropna=False, observed=True).apply(
    lambda x: x.nlargest(2, columns="v3"), meta={"id6": "Int64", "v3": "float64"}
)[["v3"]].compute()

CPU times: user 478 ms, sys: 48.1 ms, total: 526 ms
Wall time: 57.8 s


v3
id6                     
9     708485   99.930063
      1233683  99.920439
19    82918    97.625727
      625200   97.036998
33    1183098  99.414996
...                  ...
99994 168013   96.185434
99998 136085   99.405711
      197174   99.278838
99999 667180   99.555205
      290571   97.386879

[200000 rows x 1 columns]

# q9: regression v1 v2 by id2 id4

In [45]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id2", "id4", "v1", "v2"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [46]:
%%time
ddf[["id2", "id4", "v1", "v2"]].groupby(
    ["id2", "id4"], dropna=False, observed=True
).apply(
    lambda x: pd.Series({"r2": x.corr()["v1"]["v2"] ** 2}), meta={"r2": "float64"}
).compute()

CPU times: user 76.6 ms, sys: 9.08 ms, total: 85.7 ms
Wall time: 3.44 s


r2
id2   id4              
id057 1    5.092404e-04
      11   2.651509e-03
      16   2.911261e-03
      23   7.355454e-03
      31   4.390018e-04
...                 ...
id007 35   6.486901e-04
      38   9.607708e-07
      59   9.190968e-04
      90   3.711402e-04
      92   7.981009e-05

[10000 rows x 1 columns]

In [47]:
%%time
x[["id2", "id4", "v1", "v2"]].groupby(
    ["id2", "id4"], dropna=False, observed=True
).apply(
    lambda x: pd.Series({"r2": x.corr()["v1"]["v2"] ** 2}), meta={"r2": "float64"}
).compute()

CPU times: user 144 ms, sys: 13.1 ms, total: 157 ms
Wall time: 13.4 s


r2
id2   id4              
id057 1    5.092404e-04
      11   2.651509e-03
      16   2.911261e-03
      23   7.355454e-03
      31   4.390018e-04
...                 ...
id007 35   6.486901e-04
      38   9.607708e-07
      59   9.190968e-04
      90   3.711402e-04
      92   7.981009e-05

[10000 rows x 1 columns]

## q10: sum v3 count by id1:id6

In [48]:
ddf = dd.read_parquet(
    src_grp_1e7_parquet,
    columns=["id1", "id2", "id3", "id4", "id5", "id6", "v1", "v3"],
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
)

In [49]:
%%time
ddf.groupby(
    ["id1", "id2", "id3", "id4", "id5", "id6"], dropna=False, observed=True
).agg({"v3": "sum", "v1": "size"}).compute()

CPU times: user 1.79 s, sys: 778 ms, total: 2.57 s
Wall time: 22.1 s


v3  v1
id1   id2   id3          id4 id5 id6                 
id016 id016 id0000042202 15  24  5971   37.211254   1
            id0000096717 90  36  24881  68.265721   1
            id0000050660 56  2   31346  69.197638   1
            id0000006913 30  76  22749  60.054044   1
            id0000032257 19  19  41901  71.002708   1
...                                           ...  ..
id054 id002 id0000063962 43  67  18942  51.507418   1
            id0000098794 99  100 27574  96.151364   1
            id0000063695 6   90  7197    2.603732   1
            id0000078453 2   42  53619  74.687472   1
            id0000014636 83  71  67900   5.967089   1

[10000000 rows x 2 columns]

In [50]:
%%time
x.groupby(["id1", "id2", "id3", "id4", "id5", "id6"], dropna=False, observed=True).agg(
    {"v3": "sum", "v1": "size"}
).compute()

CPU times: user 1.66 s, sys: 697 ms, total: 2.36 s
Wall time: 35.9 s


v3  v1
id1   id2   id3          id4 id5 id6                 
id016 id016 id0000042202 15  24  5971   37.211254   1
            id0000096717 90  36  24881  68.265721   1
            id0000050660 56  2   31346  69.197638   1
            id0000006913 30  76  22749  60.054044   1
            id0000032257 19  19  41901  71.002708   1
...                                           ...  ..
id054 id002 id0000063962 43  67  18942  51.507418   1
            id0000098794 99  100 27574  96.151364   1
            id0000063695 6   90  7197    2.603732   1
            id0000078453 2   42  53619  74.687472   1
            id0000014636 83  71  67900   5.967089   1

[10000000 rows x 2 columns]